# **CIFAR-100**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.datasets import cifar100
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Split into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Build model function
def build_model(input_shape, num_hidden_layers=3, neurons_per_layer=128, activation='relu', optimizer='adam', dropout_rate=0.0):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))

    for _ in range(num_hidden_layers):
        model.add(Dense(neurons_per_layer, activation=activation))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(100, activation='softmax'))

    if optimizer == 'sgd':
        opt = SGD()
    elif optimizer == 'adam':
        opt = Adam()
    elif optimizer == 'rmsprop':
        opt = RMSprop()

    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train model function
def train_model(model, x_train, y_train, x_val, y_val, batch_size=32, epochs=10):
    early_stop = EarlyStopping(monitor='val_loss', patience=3)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, callbacks=[early_stop])
    return history

# Train and evaluate model
model_cifar100 = build_model(input_shape=(32, 32, 3), num_hidden_layers=3, neurons_per_layer=128, optimizer='adam', activation='relu')
history_cifar100 = train_model(model_cifar100, x_train, y_train, x_val, y_val, batch_size=32, epochs=10)

test_loss_100, test_acc_100 = model_cifar100.evaluate(x_test, y_test, verbose=2)
print(f"CIFAR-100 Test accuracy: {test_acc_100}")

# Confusion Matrix
y_pred_100 = np.argmax(model_cifar100.predict(x_test), axis=1)
cm_100 = confusion_matrix(y_test, y_pred_100)
plt.figure(figsize=(12, 10))
sns.heatmap(cm_100, annot=False, cmap='Blues', xticklabels=range(100), yticklabels=range(100))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('CIFAR-100 Confusion

# **CIFAR-10**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Split into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Build model function
def build_model(input_shape, num_hidden_layers=3, neurons_per_layer=64, activation='relu', optimizer='adam', dropout_rate=0.0):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))

    for _ in range(num_hidden_layers):
        model.add(Dense(neurons_per_layer, activation=activation))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(10, activation='softmax'))

    if optimizer == 'sgd':
        opt = SGD()
    elif optimizer == 'adam':
        opt = Adam()
    elif optimizer == 'rmsprop':
        opt = RMSprop()

    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train model function
def train_model(model, x_train, y_train, x_val, y_val, batch_size=32, epochs=5):
    early_stop = EarlyStopping(monitor='val_loss', patience=3)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, callbacks=[early_stop])
    return history

# Hyperparameter tuning
hyperparameters = {
    "epochs_list": [5, 10],
    "hidden_layers_list": [2, 4],
    "neurons_list": [128, 256],
    "batch_size_list": [32, 64],
    "optimizers": ['adam', 'rmsprop'],
    "activation_functions": ['relu', 'sigmoid']
}

best_accuracy = 0
best_model = None
best_config = {}

for epochs in hyperparameters["epochs_list"]:
    for num_layers in hyperparameters["hidden_layers_list"]:
        for neurons in hyperparameters["neurons_list"]:
            for batch_size in hyperparameters["batch_size_list"]:
                for optimizer in hyperparameters["optimizers"]:
                    for activation in hyperparameters["activation_functions"]:
                        print(f"Training with: {epochs} epochs, {num_layers} layers, {neurons} neurons, {batch_size} batch size, {optimizer} optimizer, {activation} activation")
                        model = build_model(input_shape=(32, 32, 3), num_hidden_layers=num_layers, neurons_per_layer=neurons, optimizer=optimizer, activation=activation)
                        history = train_model(model, x_train, y_train, x_val, y_val, batch_size=batch_size, epochs=epochs)

                        val_acc = max(history.history['val_accuracy'])
                        if val_acc > best_accuracy:
                            best_accuracy = val_acc
                            best_model = model
                            best_config = {'epochs': epochs, 'num_layers': num_layers, 'neurons': neurons, 'batch_size': batch_size, 'optimizer': optimizer, 'activation': activation}

print(f"Best model config: {best_config} with validation accuracy: {best_accuracy}")

# Evaluate best model on test set
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

# Confusion Matrix
y_pred = np.argmax(best_model.predict(x_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
